# Propose with Softmax

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 100 # 40
MAX_SEQUENCE_LENGTH_D = 500 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'openoffice'
METHOD = 'propose_softmax'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [9]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [10]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


72234

#### Dicionário de títulos e descrições

In [11]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 27.2 s, sys: 1.08 s, total: 28.3 s
Wall time: 27.3 s


#### Hashing bugs by buckets

In [12]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [13]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids
CPU times: user 19.7 s, sys: 10.3 ms, total: 19.7 s
Wall time: 19.7 s


#### Recovery bug ids from train

In [14]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

#### Display a random bug

In [15]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '2\n',
 'bug_status': '2\n',
 'component': '31\n',
 'creation_ts': '2004-12-12 20:19:00 +0000',
 'delta_ts': '2008-01-02 11:37:34 +0000',
 'description': 'just run file export as pdf on the attached file on page of the pdf everything after the header is shifted drastically to the upper right',
 'description_word': array([  189,   360,    24,   183,    27,   195,    21,     2,   123,
           24,    21,    61,    10,     2,   195,   918,    97,     2,
          524,     7,  3326, 16358,     4,     2,  1497,   136,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

### Generating the batch test

In [16]:
"Train ", len(baseline.dup_sets_train)

('Train ', 11043)

In [17]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = baseline.batch_iterator(baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1)
test_gen = ([valid_input_sample['title'], valid_input_pos['title'], valid_input_neg['title'], 
             valid_input_sample['description'], valid_input_pos['description'], valid_input_neg['description'],
            valid_input_sample['info'], valid_input_pos['info'], valid_input_neg['info']], valid_sim)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 33.3 ms, sys: 0 ns, total: 33.3 ms
Wall time: 32.9 ms


In [18]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_input_sample['info'].shape, valid_sim.shape

((128, 100), (128, 500), (128, 729), (128,))

### Validar entrada

In [19]:
%%time 

baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

***Title***: menu missing in basic ide
***Title***: macros no menu bar for module dialog editing window
***Description***: autotest any basic bas only verified on linux since src m the menu in the basic ide is absent
***Description***: run soffice open the tools macros organise dialogs new dialog create a new dialog click edit the window opens but it has no menu bar but only two toolbars this can be closed by ctrl w or ctrl f backing window is shown again
***similar = 1
########################
***Title***: can not insert a tab comma delimited external text csv file into a spreadsheet
***Title***: unable to insert local file
***Description***: there seems to be an issue in inserting any all types of external data specially a tab comma delimited data in a txt or other files insert external data click on external data source and select the csv file select filter i select text csv and go thru formating selecting the data columns and click ok get back to the external data windows the ok bu

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [20]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed_fasttext.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [21]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 76631'

In [22]:
# import io

# def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
#     embeddings_index = {}
#     embed_path = os.path.join(EMBED_DIR, 'crawl-300d-2M.vec')
#     f = open(embed_path, 'rb')
#     f = io.open(embed_path, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, f.readline().split())

#     vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed_fasttext.pkl'))
#     vocab_size = len(vocab) 

#     # Initialize uniform the vector considering the Tanh activation
#     embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
#     embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

#     loop = tqdm(f)
#     loop.set_description("Loading FastText")
#     for line in loop:
#         tokens = line.rstrip().split(' ')
#         embed = list(map(float, tokens[1:]))
#         word = tokens[0]
#         embeddings_index[word] = np.asarray(embed, dtype='float32')
#         loop.update(1)
#     f.close()
#     loop.close()

#     print('Total %s word vectors in FastText 42B 300d.' % len(embeddings_index))

#     loop = tqdm(total=vocab_size)
#     loop.set_description('Loading embedding from dataset pretrained')
#     i = 0
#     for word, embed in vocab.items():
#         if word in embeddings_index:
#             embedding_matrix[i] = embeddings_index[word]
#         else:
#             embedding_matrix[i] = np.asarray(embed, dtype='float32')
#         loop.update(1)
#         i+=1
#     loop.close()
#     baseline.embedding_matrix = embedding_matrix

In [23]:
def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(EMBED_DIR, 'glove.42B.300d.txt')
    f = open(embed_path, 'rb')
    #num_lines = sum(1 for line in open(embed_path, 'rb'))

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embeddings_index[word] = np.asarray(tokens[1:], dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [24]:
%%time

generating_embed(baseline, EMBED_DIR=EMBED_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1917494 word vectors in Glove 42B 300d.



CPU times: user 1min 24s, sys: 3.32 s, total: 1min 27s
Wall time: 1min 26s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [25]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [26]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable, name):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer_{}'.format(name),
                                  weights=[embeddings],
                                  embeddings_constraint=MaxNorm(max_value=1, axis=0),
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### CNN Dilated

In [27]:
from keras.constraints import max_norm
import math

def DC_CNN_Block(nb_filter, filter_length, dilation, l2_layer_reg):
    def block(block_input):        
        residual =    block_input
        
        layer_out =   Conv1D(filters=nb_filter, kernel_size=filter_length, 
                      dilation_rate=dilation, 
                      activation='linear', padding='causal', use_bias=False)(block_input) #kernel_regularizer=l2(l2_layer_reg)                    
        
        activation_out = Activation('tanh')(layer_out)
        
        skip_out =    Conv1D(1,1, activation='linear', use_bias=False)(activation_out) # use_bias=False, kernel_constraint=max_norm(1.)
        
        c1x1_out =    Conv1D(1,1, activation='linear', use_bias=False)(activation_out)
                      
        block_out =   Add()([residual, c1x1_out])
        
        return block_out, skip_out
    return block

def cnn_dilated_model(embedding_layer, title_layer, max_sequence_length):
    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput_CNND')
    embedded_sequences = embedding_layer(sequence_input)

    units = 128
    number_of_layers = 6
    
    title_input = title_layer.input
    title_layer = title_layer.output

    # Embedding layer with CNN dilated
    #la, lb = DC_CNN_Block(units,2,1,0.01)(embedded_sequences)
    la = embedded_sequences
    la_title = title_layer
    attention_layes, attention_title_layes = [], []
    filters_size = [3, 4, 5]
    number_of_filters = len(filters_size)
    for index in range(1, number_of_layers + 1):
        # Desc
        la, lb = DC_CNN_Block(units, 5, int(math.pow(2, index)), 0.01)(la)
        # Title 
        la_title, lb_title = DC_CNN_Block(units, 3, int(math.pow(2, index)), 0.01)(la_title)
        lb = Add()([lb_title, lb])
        #la = Dropout(.90)(la)
        #lb = Dropout(.90)(lb)
        attention_layes.append(lb)
        attention_title_layes.append(lb_title)

    attention_layer = Add()(attention_layes)
    attention_title_layes = Add()(attention_title_layes)
    attention_layer =   Add()([attention_layer, attention_title_layes])
    
    #layer = Add()([attention_layer, l9])
    
    layer =   Activation('tanh')(attention_layer)

    #layer =  Conv1D(1,1, activation='linear', use_bias=False)(layer)
    
    #layer = Flatten()(layer)
    layer = GlobalAveragePooling1D()(layer)
    #layer = Dropout(0.50)(layer)
    #layer = Dense(300, activation='tanh')(layer)
    #layer = GRU(150, activation='tanh', return_sequences=False)(layer)

    cnn_dilated_feature_model = Model(inputs=[sequence_input, title_input], outputs=[layer], name = 'FeatureCNNDilatedGenerationModel') # inputs=visible
    return cnn_dilated_feature_model

### CNN with filter 3,4,5

In [28]:
import keras
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, AveragePooling1D

def cnn_model(embedding_layer, max_sequence_length):

    sequence_input = Input(shape=(max_sequence_length,), name='Feature_BugInput')
    #sequence_input = Input(shape=(None,), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    # best combination filter (3, 4, 5) e 128 e 256
    convs = []
    filter_sizes = [3, 4, 5]
    n_filters = 64

    for index, filter_size in enumerate(filter_sizes):
        l_conv = Conv1D(filters=n_filters, kernel_size=filter_size)(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=filter_size)(l_conv) # index+1
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    #conv = Conv1D(filters=n_filters * 3, kernel_size=3)(l_merge)
    layer = GlobalAveragePooling1D()(l_merge)
    #layer = Flatten()(l_merge)
    layer = Dense(300, activation='tanh')(layer)
    #layer = LeakyReLU()(layer)

    cnn_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureCNNGenerationModel') # inputs=visible

    return cnn_feature_model

### Bi-LSTM

In [29]:
from keras.layers import Dense, Input, LSTM, GRU, Dropout, Bidirectional, GlobalAveragePooling1D, Permute, Dot

def bilstm_model(embedding_layer, max_sequence_length):
    number_lstm_units = 50
    rate_drop_lstm = 0
    recurrent_dropout = 0

    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    #sequence_input = Input(shape=(None, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Creating LSTM Encoder
#     lstm_layer = Bidirectional(LSTM(number_lstm_units, return_sequences=True), # dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm 
#                                merge_mode='ave')

    left_layer = LSTM(number_lstm_units, return_sequences=True)(embedded_sequences)
    right_layer = LSTM(number_lstm_units, return_sequences=True, go_backwards=True)(left_layer)
    
    lstm_layer = Add()([left_layer, right_layer])
    
    #lstm_layer = TimeDistributed(Dense(1))(lstm_layer)
    layer = Flatten()(lstm_layer)
    #layer = GlobalAveragePooling1D()(layer)
    layer = Dense(300, activation='tanh')(layer)

    lstm_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureLstmGenerationModel') # inputs=visible

    return lstm_feature_model

### MLP

In [30]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    for units in [64, 32]:
        layer = Dense(units, activation='tanh', kernel_initializer='random_uniform')(info_input)
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [31]:
from keras import backend as K
import tensorflow as tf

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance, axis=-1, keepdims=False)

def euclidean_distance(vects):
    x, y = vects
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))
    # Normalize https://stats.stackexchange.com/questions/53068/euclidean-distance-score-and-similarity
    distance = K.constant(1) / (K.constant(1) + distance)
    return K.mean(distance, keepdims=False)
    #return K.mean(distance, axis=-1, keepdims=False)

# https://jdhao.github.io/2017/03/13/some_loss_and_explanations/
def triplet_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, pos - neg + margin))

def custom_margin_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, margin - pos + neg), keepdims=False)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

def pos_distance(y_true, y_pred):
    return y_pred[0]

def neg_distance(y_true, y_pred):
    return y_pred[1]

def stack_tensors(vects):
    return K.stack(vects, axis=-1)

#### Propose

In [32]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum, Subtract, Average
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    #bug_d_feat = desc_feature_model([bug_d, bug_t])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_t_feat = GlobalAveragePooling1D()(bug_t_feat)
    
#     encoded_t_1a, encoded_t_1b  = residual_bug()(bug_t_feat)
#     encoded_d_1a, encoded_d_1b  = residual_bug()(bug_d_feat)
#     bug_t_feat = encoded_t_1a
#     bug_d_feat = encoded_d_1a
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    #bug_feature_output, bug_feature_output_1b = residual_bug()(bug_feature_output)
    #bug_feature_output_1a = Dropout(.5)(bug_feature_output_1a)
    #bug_feature_output, bug_feature_output_2b = residual_bug()(bug_feature_output_1a)
    
    #bug_feature_output = Add()([bug_feature_output_1b, bug_feature_output_2b])
    #bug_feature_output = BatchNormalization()(bug_feature_output)
    #bug_feature_output = Activation('relu')(bug_feature_output)
#     bug_feature_output = Dropout(.75)(bug_feature_output)
#     shape_size = K.int_shape(bug_feature_output)[1]
#     bug_feature_output = Dense(shape_size, activation='linear', use_bias=False)(bug_feature_output)
#     bug_feature_output = Dropout(.33)(bug_feature_output)
#     bug_feature_output = Dense(100)(bug_feature_output)
    
    #bug_feature_output  = residual_bug()(bug_feature_output)
    #bug_feature_output = BatchNormalization()(bug_feature_output)
    #     encoded_2a, encoded_2b  = residual_bug()(encoded_1a)
    
    #     bug_feature_output = Add()([encoded_1b, encoded_2b])
    #     bug_feature_output = Activation('tanh')(bug_feature_output)
    
    # Bug representation layer
    # bug_feature_output = Dense(300, activation='tanh')(bug_feature_output)
    
    bug_feature_model = Model(inputs=[bug_t, bug_d, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [33]:
from keras.layers import Concatenate

def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    
    # Distance
    bug_anchor_pos = Concatenate()([encoded_anchor, encoded_positive])
    bug_anchor_neg = Concatenate()([encoded_anchor, encoded_negative])
    positive_d = Dense(2, activation='softmax')(bug_anchor_pos)
    negative_d = Dense(2, activation='softmax')(bug_anchor_neg)

    # Loss function only works with a single output
#     output = Lambda(
#         lambda vects: stack_tensors(vects),
#         name='stack-distances',
#         output_shape=(2, 1)
#     )([positive_d, negative_d])

    #similarity_layer = Average()([positive_d, negative_d])

    similarity_model = Model(inputs = inputs, outputs = [positive_d, negative_d], name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    optimizer = Adam(lr=1e-3 * decay_lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer=optimizer, loss=['binary_crossentropy', 'binary_crossentropy'])

    return similarity_model

In [34]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def to_onehot(train_sim):
    onehot_encoder = OneHotEncoder(sparse=False)
    train_sim = train_sim.reshape(len(train_sim), 1)
    train_sim = onehot_encoder.fit_transform(train_sim)
    return train_sim

In [35]:
# %%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False, name='desc')
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False, name='title')

# Feature models
'''
    cnn_dilated_model
    arcii_model
    cnn_model
    lstm_model
    bilstm_model
'''
title_feature_model = bilstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
desc_feature_model = cnn_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')
encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()


'''
    Configuration
'''
epochs = 100
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = baseline.batch_iterator(baseline.train_data, baseline.dup_sets_train, bug_train_ids, batch_size, 1)
    train_batch = [train_input_sample['title'], train_input_sample['description'], train_input_sample['info'],
                   train_input_pos['title'], train_input_pos['description'], train_input_pos['info'], 
                   train_input_neg['title'], train_input_neg['description'], train_input_neg['info']]
    
    # binary encode
    #train_sim_inverse = to_onehot(train_sim[::-1])
    train_sim = to_onehot(train_sim)
    
    h = similarity_model.train_on_batch(x=train_batch, y=[train_sim, train_sim])
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, bug_train_ids)
        print("Epoch: {} Loss positive: {:.2f}, Loss negative: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h[1], recall))
    else:
        print("Epoch: {} Loss positive: {:.2f}, Loss negative: {:.2f}".format(epoch+1, h[0], h[1]))
    
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 729)          0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
desc_in (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
info_pos (InputLayer)           (None, 729)          0                                            
__________________________________________________________________________________________________
title_pos 

Epoch: 52 Loss positive: 1.45, Loss negative: 0.72
Epoch: 53 Loss positive: 1.42, Loss negative: 0.72
Epoch: 54 Loss positive: 1.39, Loss negative: 0.69
Epoch: 55 Loss positive: 1.40, Loss negative: 0.70
Epoch: 56 Loss positive: 1.41, Loss negative: 0.70
Epoch: 57 Loss positive: 1.41, Loss negative: 0.71
Epoch: 58 Loss positive: 1.40, Loss negative: 0.70
Epoch: 59 Loss positive: 1.39, Loss negative: 0.69
Epoch: 60 Loss positive: 1.41, Loss negative: 0.71
Epoch: 61 Loss positive: 1.40, Loss negative: 0.71
Epoch: 62 Loss positive: 1.42, Loss negative: 0.71
Epoch: 63 Loss positive: 1.41, Loss negative: 0.71
Epoch: 64 Loss positive: 1.38, Loss negative: 0.69
Epoch: 65 Loss positive: 1.37, Loss negative: 0.69
Epoch: 66 Loss positive: 1.39, Loss negative: 0.69
Epoch: 67 Loss positive: 1.37, Loss negative: 0.68
Epoch: 68 Loss positive: 1.41, Loss negative: 0.70
Epoch: 69 Loss positive: 1.40, Loss negative: 0.70
Epoch: 70 Loss positive: 1.43, Loss negative: 0.71
Epoch: 71 Loss positive: 1.38, 

In [36]:
_[:20]

['108544:111059,109674,108379,109366|108379:0.4709373712539673,107779:0.45197659730911255,105671:0.44646257162094116,104574:0.4323633313179016,104576:0.4323633313179016,105272:0.43221497535705566,106168:0.4292343258857727,116229:0.42878657579421997,92365:0.42761605978012085,113113:0.42485421895980835,108492:0.4248132109642029,108529:0.4241333603858948,107794:0.4237405061721802,104173:0.4235216975212097,98647:0.42267364263534546,106234:0.4225043058395386,110928:0.42099452018737793,107267:0.41950303316116333,93318:0.41931378841400146,46957:0.4191850423812866,106462:0.4180976152420044,96429:0.4180328845977783,110322:0.4166584014892578,115557:0.41455841064453125,102712:0.41451114416122437,93171:0.4144558906555176,105961:0.41392362117767334,110323:0.4122380018234253,102450:0.4118548035621643',
 '109674:108544,111059,108379,109366|95277:0.38510167598724365,108529:0.38058340549468994,93288:0.37393200397491455,107845:0.362984836101532,90533:0.3591517210006714,92543:0.3564755916595459,114153:0.

In [37]:
# recall, exported_rank, debug = experiment.evaluate_validation_test(experiment, retrieval, verbose, 
#                                                         encoded_anchor, issues_by_buckets, evaluate_validation_test)
# test_vectorized, queries_test_vectorized, annoy, X_test, distance_test, indices_test = debug
# "recall@25 last epoch:", recall

### Retrieval evaluation

In [38]:
print("Total of queries:", len(retrieval.test))

Total of queries: 2086


#### Getting the model trained

In [39]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'propose_softmax_feature_100epochs_64batch(openoffice)'

In [40]:
model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [41]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 729)          0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
desc_in (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
FeatureMlpGenerationModel (Mode (None, 300)          219000      info_in[0][0]                    
__________________________________________________________________________________________________
FeatureLst

In [42]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, bug_train_ids)

In [43]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/exported_rank_propose_softmax.txt'

In [44]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [45]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.36,
 '2 - recall_at_10': 0.44,
 '3 - recall_at_15': 0.48,
 '4 - recall_at_20': 0.52,
 '5 - recall_at_25': 0.55}